In [3]:
# # !pip install webrtcvad
# !pip install pygame

In [2]:
# app.py
from flask import Flask, render_template, jsonify

app = Flask(__name__)

# Initialize the file number
ai_file_no = 1

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/mute', methods=['POST'])
def mute_audio():
    global mute
    mute = True
    return jsonify({'status': 'Muted'})

@app.route('/unmute', methods=['POST'])
def unmute_audio():
    global mute
    mute = False
    return jsonify({'status': 'Unmuted'})

@app.route('/ai_speak', methods=['POST'])
def ai_speak():
    global ai_file_no

    # Define the range for AI files
    if ai_file_no < 1 or ai_file_no > 24:
        ai_file_no = 1  # Reset to the first file if out of range

    selected_file = f'Ai-{ai_file_no}.wav'
    ai_file_no += 1  # Increment to the next file number

    return jsonify({'response': f'/static/ai_responses/{selected_file}'})

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/May/2024 19:07:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2024 19:07:21] "POST /ai_speak HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2024 19:07:21] "GET /static/ai_responses/Ai-1.wav HTTP/1.1" 206 -
127.0.0.1 - - [26/May/2024 19:07:21] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [26/May/2024 19:07:22] "POST /unmute HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2024 19:07:22] "POST /mute HTTP/1.1" 200 -
